In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(101)

In [4]:
win_counts = np.array([
    [0.0, 14.0, 29.5, 20.5, 3.5, 5.5],
    [754.0, 0.0, 94.5, 205.5, 115.5, 91.0],
    [738.5, 673.5, 0.0, 222.5, 199.0, 303.5],
    [747.5, 562.5, 545.5, 0.0, 396.5, 247.5],
    [764.5, 652.5, 569.0, 371.5, 0.0, 250.0],
    [762.5, 677.0, 464.5, 520.5, 518.0, 0.0]
])

# win_rates = wins_to_winrates(win_counts)
# print(win_rates)
# print()
# elos = winrates_to_elos(win_rates)
# print(elos)

In [5]:
def wins_to_winrates(win_counts):
    total_games = win_counts[0, 1] + win_counts[1, 0]
    win_rates = win_counts / total_games if total_games > 0 else np.zeros_like(win_counts, dtype=float)
    return win_rates

In [6]:
def winrates_to_odds(win_rates):
    odds = win_rates / (1 - win_rates)
    return odds

In [7]:
# This is scuffed, doesn't really work. 
# ELO is not win-order invariant
def winrates_to_elos(win_rates, initial_rating=0, k_factor=32, max_iterations=1000, tolerance=1e-4):
    """
    k_factor: Maximum possible adjusment per game 
    max_iterations: Maximum number of iterations to perform
    tolerance: the algorithm stops if the max change in any rating is below this threshold
    """
    
    num_bots = win_rates.shape[0]
    ratings = np.full(num_bots, initial_rating, dtype=float)
    temp_ratings = ratings.copy()

    for _ in range(max_iterations):
        max_change = 0
        for i in range(num_bots):
            for j in range(num_bots):
                if i != j:
                    expected = 1 / (1 + 10 ** ((ratings[j] - ratings[i]) / 400)) # s = 400 (scaling factor)
                    actual = win_rates[j, i]

                    rating_change = k_factor * (actual - expected) 
                    temp_ratings[i] += rating_change # R -> R + K(S - E)
                    max_change = max(max_change, abs(rating_change))
        
        ratings = temp_ratings.copy() # Update ratings

        if max_change < tolerance: # changes star of large, get small
            break

    return ratings



In [8]:
def parse_game_log(file_path):
    game_logs = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove any leading/trailing whitespace
            parts = line.split()  # Split the line into parts
            
            if len(parts) == 3:  # Ensure the line is correctly formatted
                player_i = int(parts[0])  # First part: index of player i
                player_j = int(parts[1])  # Second part: index of player j
                result = int(parts[2])  # Third part: game result (-1, 0, 1)

                # Create a dictionary for this game log
                game_log = {
                    'player_i': player_i,
                    'player_j': player_j,
                    'result': result
                }

                # Append the dictionary to the list
                game_logs.append(game_log)

    return game_logs


In [ ]:
def update_elo(ratings, player_i, player_j, result, k=32):
    # Calculate expected scores
    E_i = 1 / (1 + 10 ** ((ratings[player_j] - ratings[player_i]) / 400))
    E_j = 1 / (1 + 10 ** ((ratings[player_i] - ratings[player_j]) / 400))

   
    if result == 0:  # Player i wins
        S_i, S_j = 1, 0
    elif result == 1:  
        S_i, S_j = 0, 1
    else:  # Draw
        S_i, S_j = 0.5, 0.5

    # Update ratings
    ratings[player_i] += k * (S_i - E_i)
    ratings[player_j] += k * (S_j - E_j)

In [ ]:
def compute_elo_from_logs(game_logs):

    # Initialize ratings (assuming no player indices beyond what's seen in the logs)
    ratings = {}
    for log in game_logs:
        ratings[log['player_i']] = 1500
        ratings[log['player_j']] = 1500

    np.random.shuffle(game_logs)

    # Process each game
    for log in game_logs:
        update_elo(ratings, log['player_i'], log['player_j'], log['result'])

    return ratings


In [ ]:
logs_file_apth = "need file"
#will need to iterate over logs

game_logs = parse_game_log(logs_file_apth)

elos = compute_elo_from_logs(game_logs)

print(elos)